In [18]:
import pandas            as pd 
import matplotlib.pyplot as plt
# import shap 

from joblib import load
from sklearn.model_selection   import train_test_split
from sklearn.feature_selection import RFE

import pickle 

%matplotlib inline

model_lr = load('import/LR_l1_c10.joblib')
#model_lgbm = load('import/LGBM.joblib')
#model_optimal_xgb = load('optimal_xgb.joblib')
#model_rf = load('rf_pluslight.joblib')

##### Import of tables into dataframes
data   = pd.read_pickle('import/20221112_table_feature_matrix.pkl').reset_index()
target = pd.read_pickle('import/20221119_table_target.pkl').reset_index()

data = data.drop(['Num_Acc'], axis = 1)
target= target.drop(['Num_Acc'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 1)
# target.head()



c:\Users\possa\miniforge3\envs\datasc01\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.2.0 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
# Score du modèle
lr_score_train= model_lr.score(X_train, y_train)
print('Score du LR sur l entrainement:{:.4f}'.format(lr_score_train))

lr_score_test= model_lr.score(X_test, y_test)
print('Score du LR sur l echantillon de test:{:.4f}'.format(lr_score_test))


Score du LR sur l entrainement:0.7254
Score du LR sur l echantillon de test:0.7241


In [3]:
from fonctions import * 
cutoff , model_pred_test= get_optimized_cutoff(model_lr,X_test, y_test, 0.8)


              precision    recall  f1-score   support

           0       0.68      0.93      0.78     96536
           1       0.80      0.40      0.54     70775

    accuracy                           0.71    167311
   macro avg       0.74      0.67      0.66    167311
weighted avg       0.73      0.71      0.68    167311



In [4]:
# print(cutoff)


In [5]:
df_results = recall_features(X_test, y_test, model_pred_test).reset_index().drop(['index'], axis=1)
print(df_results)

                        feature  precision    recall  accuracy
0                     infra_7.0   0.906250  0.557692  0.701149
1       populationGrp_Métropole   0.886486  0.036219  0.772125
2                      situ_6.0   0.857143  0.142857  0.750000
3                       lum_3.0   0.828025  0.742297  0.723232
4            catv_train_exist_1   0.823529  0.893617  0.770492
5                      catr_3.0   0.815013  0.666559  0.696616
6                       col_2.0   0.805268  0.146877  0.749395
7                       atm_7.0   0.793136  0.678077  0.681416
8                      catr_2.0   0.786405  0.388718  0.658742
9    populationGrp_Petite Ville   0.780526  0.318905  0.609914
10                      col_4.0   0.776224  0.088165  0.795706
11                    nbvGrp_4+   0.701667  0.067167  0.748277
12  populationGrp_Ville Moyenne   0.693050  0.038640  0.680412
13   populationGrp_Grande Ville   0.554770  0.016483  0.706649
14             catv_EPD_exist_1        NaN  0.000000  0

c:\Users\possa\Documents\DataScienTEST\Accidents routiers en France\fonctions.py:68: RuntimeWarning: invalid value encountered in longlong_scalars
  precision_feature = vrais_positifs / (vrais_positifs+faux_positifs)


In [6]:
df_results[df_results['precision']>0.799]

,feature,precision,recall,accuracy
0,infra_7.0,0.906250,0.557692,0.701149
1,populationGrp_Métropole,0.886486,0.036219,0.772125
2,situ_6.0,0.857143,0.142857,0.750000
3,lum_3.0,0.828025,0.742297,0.723232
4,catv_train_exist_1,0.823529,0.893617,0.770492
5,catr_3.0,0.815013,0.666559,0.696616
6,col_2.0,0.805268,0.146877,0.749395


In [8]:
rfe_model=RFE(estimator=model_lr, n_features_to_select= 10)
rfe_model.fit(X_train, y_train)

c:\Users\possa\miniforge3\envs\datasc01\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\possa\miniforge3\envs\datasc01\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\possa\miniforge3\envs\datasc01\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\possa\miniforge3\envs\datasc01\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d arr

RFE(estimator=LogisticRegression(C=10, penalty='l1', solver='liblinear'),
    n_features_to_select=10)

In [39]:
# Score d'entrainement
rfe_score_train= rfe_model.score(X_train, y_train)

# SCore pour comparer et valider la robustesse du modèle
rfe_score_test = rfe_model.score(X_test, y_test)

print('Score RFE train: {:.4f}'.format(rfe_score_train))
print('Score RFE test : {:.4f}'.format(rfe_score_test))
print('Meme score que la LR -> donc modèle robuste')

Score RFE train: 0.7022
Score RFE test : 0.7006
Meme score que la LR -> donc modèle robuste


In [38]:
print('Les colonnes sélectionnées sont: ')
pd.DataFrame(X_train.loc[:,rfe_model.support_].columns).rename( columns={0:'Colonne selectionnée'}) 


Les colonnes sélectionnées sont: 


,Colonne selectionnée
0,catv_EPD_exist_1
1,catv_train_exist_1
2,catr_3.0
3,col_2.0
4,col_4.0
5,situ_6.0
6,populationGrp_Grande Ville
7,populationGrp_Métropole
8,populationGrp_Petite Ville
9,populationGrp_Ville Moyenne
